In [1]:
from wrapper.inference import AtomicPartitionInference
from wrapper.implementations.mattersim import MatterSimModelAdapter

from mattersim.forcefield import MatterSimCalculator

import torch
import numpy as np

from ase.io import read

from tqdm import tqdm

2025-03-26 12:02:25.782 | INFO     | mattersim.forcefield.potential:from_checkpoint:877 - Loading the pre-trained mattersim-v1.0.0-1M.pth model


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
mattersim_partition_inference = AtomicPartitionInference(MatterSimModelAdapter(device=device, num_message_passing=4))

Loading the pre-trained mattersim-v1.0.0-1M.pth model


/home/jaeheonshim/georgia_tech/research/atomic-graph-partitioning/wrapper/implementations/mattersim.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.

In [4]:
def get_mattersim_benchmark(atoms):
    mattersim_calc = MatterSimCalculator(compute_stress=False)
    atoms.calc = mattersim_calc

    return {
        "energy": atoms.get_potential_energy(),
        "forces": atoms.get_forces()
    }

In [5]:
atoms = read('datasets/test.xyz')

In [6]:
MATTERSIM_ITERATIONS = 1

benchmark_forces = []

for _ in tqdm(range(MATTERSIM_ITERATIONS)):
    benchmark = get_mattersim_benchmark(atoms)
    benchmark_forces.append(benchmark["forces"])

benchmark_forces = np.mean(benchmark_forces, axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen

In [7]:
result_forces = []
for _ in range(MATTERSIM_ITERATIONS):
    result = mattersim_partition_inference.run(atoms, desired_partitions=60)
    result_forces.append(result["forces"])

result_forces = np.mean(result_forces, axis=0)


Beginning partitioned inference
- Number of atoms                :        426
- Desired number of partitions   :         60
- Number of partitions per batch :          1

Partitioning graph...
Partitioning complete! Created 60 partitions. Average size of partition: 426.0
Starting inference...


100%|██████████| 60/60 [00:54<00:00,  1.09it/s]

Inference complete!


In [15]:
error_ratio =  np.linalg.norm(benchmark_forces - result_forces, ord=1, axis=1) / np.linalg.norm(benchmark_forces, ord=1, axis=1)

positions = atoms.positions
pos_min = np.min(positions, axis=0)
pos_max = np.max(positions, axis=0)
positions_normalized = (positions - pos_min) / (pos_max - pos_min)

In [16]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=positions_normalized[:, 0],
    y=positions_normalized[:, 1],
    z=positions_normalized[:, 2],
    mode='markers',
    marker=dict(
        size=6,
        color=error_ratio,
        colorscale='plasma',
        opacity=0.8,
        colorbar=dict(title="Normalized Error")
    )
)])

fig.update_layout(
    title="Atomic Force Normalized Error",
    height=800,
    scene=dict(
        xaxis_title="X (normalized)",
        yaxis_title="Y (normalized)",
        zaxis_title="Z (normalized)"
    )
)

fig.show()